# Decimation
- OFF file writing needs to be improved if that function is to be used at all (per line for loop writing to file is very inefficient in python)
- Decimation process requires the mesh to written to an OFF file, meshlabserver reads from that OFF file and outputs to a new OFF file, then the decimate function reads from the new OFF file. Results in several redundant and extremely expensive steps.

## Helper Functions

In [1]:
import os, contextlib
import pathlib
import subprocess

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [2]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="None",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        if folder == "None":
            file_loc = pathlib.Path.cwd()
            
        else:
            file_loc = pathlib.Path.cwd() / folder
            
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

## Decimate Function

In [3]:
# create a temp folder if doesn't already exist
import os
folder_name = "decimation_temp"
directory = "./" + str(folder_name)
if not os.path.exists(directory):
    os.makedirs(directory)

In [4]:
import trimesh

def decimate_mesh(vertices,faces,segment_id,current_folder):
    #write the file to the temp folder
#     input_file_base = write_Whole_Neuron_Off_file(vertices, faces, segment_id, folder=current_folder)
    input_file_base = os.path.join(current_folder, f'neuron_{segment_id}')
    trimesh.Trimesh(vertices=vertices, faces=faces).export(input_file_base+".off", )
    output_file = input_file_base + "_decimated"
    
    script_name = "decimation_meshlab.mls"
    meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name
    
    meshlab_fix_manifold_path_specific_mls(
        input_path_and_filename=input_file_base + ".off",
        output_path_and_filename=output_file + ".off",
        meshlab_script=meshlab_script_path_and_name
    )
    
    #read in the output mesh and return the vertices and faces
    current_mesh = trimesh.load_mesh(output_file + '.off')
    
    #check if file exists and then delete the temporary decimated mesh filess
    if os.path.exists(input_file_base + ".off"):
        os.remove(input_file_base + ".off")
    if os.path.exists(output_file + ".off"):
        os.remove(output_file + ".off")
 
    return current_mesh.vertices, current_mesh.faces

## Populate

In [5]:
import datajoint as dj
import os

segmentation_m65 = 1
segmentation_m65_str = f'{segmentation_m65:02d}'

# External store paths + ensure the directories exist. For new segmentations create a subfolder.
external_store_basepath = '/mnt/dj-stor01/platinum/minnie65'
external_segmentation_path = os.path.join(external_store_basepath, segmentation_m65_str)
external_mesh_path = os.path.join(external_segmentation_path, 'meshes')
external_decimated_mesh_path = os.path.join(external_segmentation_path, 'decimated_meshes')

if os.path.exists(external_store_basepath):
  # External filepath referrencing.
  dj.config['stores'] = {
    'minnie65': {
      'protocol': 'file',
      'location': external_store_basepath,
      'stage': external_store_basepath
    },
    'meshes': {
      'protocol': 'file',
      'location': external_mesh_path,
      'stage': external_mesh_path
    },
    'decimated_meshes': {
      'protocol': 'file',
      'location': external_decimated_mesh_path,
      'stage': external_decimated_mesh_path
    }
  }

# Enable experimental datajoint features
# These flags are required by 0.12.0+ (for now).
dj.config['enable_python_native_blobs'] = True
dj.errors._switch_filepath_types(True)
dj.errors._switch_adapted_types(True)

# Adapters
import numpy as np
import h5py
import os

from collections import namedtuple


class MeshAdapter(dj.AttributeAdapter):
    # Initialize the correct attribute type (allows for use with multiple stores)
    def __init__(self, attribute_type, has_version=False):
        self.attribute_type = attribute_type
        self.has_version = has_version
        super().__init__()

    attribute_type = '' # this is how the attribute will be declared
    has_version = False # used for file name recognition

    TriangularMesh = namedtuple('TriangularMesh', ['segment_id', 'version', 'vertices', 'faces'])
    
    def put(self, filepath):
        # save the filepath to the mesh
        filepath = os.path.abspath(filepath)
        assert os.path.exists(filepath)
        return filepath

    def get(self, filepath):
        # access the h5 file and return a mesh
        assert os.path.exists(filepath)

        with h5py.File(filepath, 'r') as hf:
            vertices = hf['vertices'][()].astype(np.float64)
            faces = hf['faces'][()].reshape(-1, 3).astype(np.uint32)
        
        segment_id = os.path.splitext(os.path.basename(filepath))[0]

        version = None
        if self.has_version:
            segment_id, version = segment_id.split('_')
            version = int(version)
        
        return self.TriangularMesh(
            segment_id=int(segment_id),
            version=version,
            vertices=vertices,
            faces=faces
        )

# instantiate for use as a datajoint type
mesh = MeshAdapter('filepath@meshes')
decimated_mesh = MeshAdapter('filepath@decimated_meshes', has_version=True)


# Virtual module accessors
minnie = dj.create_virtual_module('minnie', 'microns_minnie65_01', add_objects={'mesh': mesh, 'decimated_mesh': decimated_mesh}) # virtual module with the adapted attribute for mesh access from .h5 files

Connecting celiib@10.28.0.34:3306


In [6]:
@minnie.schema
class Decimation(dj.Computed):
    definition = minnie.Decimation.describe(printout=False)
    
    # Creates hf file at the proper location, returns the filepath of the newly created file
    @classmethod
    def make_file(cls, segment_id, version, vertices, faces):
        """Creates hf file at the proper location, returns the filepath of the newly created file"""
        
        assert vertices.ndim == 2 and vertices.shape[1] == 3
        assert faces.ndim == 2 and faces.shape[1] == 3

        filename = f'{segment_id}_{version}.h5'
        filepath = os.path.join(external_decimated_mesh_path, filename)
        with h5py.File(filepath, 'w') as hf:
            hf.create_dataset('segment_id', data=segment_id)
            hf.create_dataset('version', data=version)
            hf.create_dataset('vertices', data=vertices)
            hf.create_dataset('faces', data=faces)

        return filepath

    @classmethod
    def make_entry(cls, segment_id, version, decimation_ratio, vertices, faces):
        key = dict(
            segment_id=segment_id,
            version=version,
            decimation_ratio=decimation_ratio,
            n_vertices=len(vertices),
            n_faces=len(faces)
        )

        filepath = cls.make_file(segment_id, version, vertices, faces)

        cls.insert1(dict(key, mesh=filepath))

    key_source = minnie.Mesh - minnie.DecimationError & 'n_vertices < 1000000'

    def make(self, key):
        mesh = (minnie.Mesh & key).fetch1('mesh')
        
        version = 0
        decimation_ratio = 0.25 # Only the value in the .mls can change the ratio though.
        
        new_vertices, new_faces = decimate_mesh(mesh.vertices, mesh.faces, key['segment_id'], folder_name)

        try:
            self.make_entry(
                    segment_id=key['segment_id'],
                    version=version,
                    decimation_ratio=decimation_ratio,
                    vertices=new_vertices,
                    faces=new_faces
            )
        except Exception as e:
            key['version'] = version
            key['decimation_ratio'] = decimation_ratio
            minnie.DecimationError.insert1(dict(key, log=str(e)))
            raise e

In [7]:
Decimation.populate(suppress_errors=True)

xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i decimation_temp/neuron_76780546973661652.off -o decimation_temp/neuron_76780546973661652_decimated.off -s /notebooks/Platinum_Blender/decimation_meshlab.mls


DuplicateError: Duplicate entry '\xAA5v\x86\xA7_\xBAX\xC9\x87B\x8E\x16\xB1\x85\x0B' for key 'PRIMARY'

In [ ]:
minnie.Decimation()

In [ ]:
minnie.DecimationError()